In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-patient-precondition-dataset/covid.csv
/kaggle/input/covid19-patient-precondition-dataset/Description.xlsx
/kaggle/input/covid19-patient-precondition-dataset/Catalogs.xlsx


In [6]:
from datetime import datetime
# datetime.strptime('2012-07-12', '%Y-%m-%d').timetuple()
Df = pd.read_csv('/kaggle/input/covid19-patient-precondition-dataset/covid.csv').sample(frac=1)
Df = Df[Df['covid_res'] == 1]
Df['died'] = (Df['date_died'].astype('string') != '9999-99-99').astype(int)
Df['date_symptoms'] = pd.to_datetime(Df['date_symptoms'] ,format='%d-%m-%Y')
Df['entry_date'] = pd.to_datetime(Df['entry_date'],format='%d-%m-%Y')

Df['time_to_admission'] = (Df['entry_date'] - Df['date_symptoms']).dt.days
Df


,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu,died,time_to_admission
114027,124305,1,1,2020-05-28,2020-05-28,9999-99-99,97,2,23,2,...,2,2,2,2,2,2,1,97,0,0
49612,1c82e7,2,1,2020-06-01,2020-05-29,9999-99-99,97,2,68,97,...,2,2,1,2,2,2,1,97,0,3
30182,1b520f,2,2,2020-06-20,2020-06-12,20-06-2020,2,2,63,97,...,2,2,1,2,2,99,1,2,1,8
11091,0adc30,2,1,2020-05-19,2020-05-16,9999-99-99,97,2,33,97,...,2,2,2,2,2,2,1,97,0,3
197231,066b1a,1,1,2020-05-12,2020-05-03,9999-99-99,97,2,35,2,...,2,2,2,2,2,99,1,97,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137807,06f0b5,2,2,2020-04-28,2020-04-17,9999-99-99,2,1,58,97,...,2,2,1,2,2,2,1,2,0,11
205034,13f138,2,1,2020-06-18,2020-06-15,9999-99-99,97,2,30,97,...,2,2,2,2,1,1,1,97,0,3
17757,04edc0,1,1,2020-06-02,2020-05-30,9999-99-99,97,2,47,2,...,2,2,1,2,2,99,1,97,0,3
66319,1da557,2,1,2020-06-18,2020-06-16,9999-99-99,97,2,48,97,...,2,2,1,2,1,2,1,97,0,2


In [7]:
ord_cols = {'id','entry_date','date_symptoms','date_died','died','time_to_admission','age'}
cat_cols = []
for col in Df.columns:
    if col in ord_cols:
        continue
    cat_cols.append(col)
print(cat_cols)
df_ohe = pd.get_dummies(Df,columns=cat_cols,drop_first=True)
df_ohe = df_ohe.drop(columns = ['id','entry_date','date_symptoms','date_died'])
df_ohe

['sex', 'patient_type', 'intubed', 'pneumonia', 'pregnancy', 'diabetes', 'copd', 'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']


,age,died,time_to_admission,sex_2,patient_type_2,intubed_2,intubed_97,intubed_99,pneumonia_2,pneumonia_99,...,obesity_98,renal_chronic_2,renal_chronic_98,tobacco_2,tobacco_98,contact_other_covid_2,contact_other_covid_99,icu_2,icu_97,icu_99
114027,23,0,0,0,0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
49612,68,0,3,1,0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
30182,63,1,8,1,1,1,0,0,1,0,...,0,1,0,1,0,0,1,1,0,0
11091,33,0,3,1,0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
197231,35,0,9,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137807,58,0,11,1,1,1,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0
205034,30,0,3,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
17757,47,0,3,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,1,0
66319,48,0,2,1,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0


In [8]:
from sklearn.model_selection import train_test_split as tts
from imblearn.over_sampling import RandomOverSampler 


X_train, X_test, y_train, y_test = tts(
    df_ohe.drop(columns = ['died']), df_ohe['died'], test_size=0.20, random_state=42)
# ros = RandomOverSampler(random_state=42)
# X_train, y_train = ros.fit_resample(X_train, y_train)

In [9]:
np.sum((y_res == 0).astype(int))

NameError: name 'y_res' is not defined

In [10]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report as cr
clf = RFC()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(cr(y_test,y_pred))
y_pred = clf.predict(X_train)
print(cr(y_train,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93     38743
           1       0.53      0.46      0.49      5389

    accuracy                           0.88     44132
   macro avg       0.73      0.70      0.71     44132
weighted avg       0.88      0.88      0.88     44132

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    154793
           1       0.90      0.82      0.86     21732

    accuracy                           0.97    176525
   macro avg       0.94      0.90      0.92    176525
weighted avg       0.97      0.97      0.97    176525



In [ ]:
import seaborn as sns

sns.violinplot(y_test, X_test['age'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC